<table style="border:none; border-collapse:collapse; cellspacing:0; cellpadding:0">
<tr>
    <td width=30% style="border:none">
        <center>
            <img src="../images/iapau_icon.png" width="30%"/><br>
            <a href="https://iapau.org/">Association IA Pau</a><br>
            <a href="https://iapau.org/events/festival/">Festival IAPau 7</a>
        </center>
    </td>
    <td style="border:none">
        <center>
            <h1>Atelier - Agentic RAG</h1>
            <h2>Data Acquisition</h2>
        </center>
    </td>
    <td width=20% style="border:none">
    </td>
</tr>
</table>

---

## 📋 Table des matières

1. [**Téléchargement des données SEC EDGAR**](#sec-edgar)
   - Acquisition des dépôts financiers de Nvidia
   - Téléchargement des rapports 10-K, 10-Q, 8-K et DEF 14A

2. [**Création du jeu de données relationnel**](#donnees-relationnelles)
   - Génération d'un fichier CSV avec les indicateurs financiers
   - Structure des données pour l'agent Analyst

- [**Prochaines étapes**](#prochaines-etapes)
   - Résumé des accomplissements et orientation vers la Phase 1

---

In [9]:
import os
import pandas as pd
from getpass import getpass
from pathlib import Path

# Data Acquisition
from sec_edgar_downloader import Downloader

print("Toutes les bibliothèques ont été importées avec succès !")

Toutes les bibliothèques ont été importées avec succès !


<a id="sec-edgar"></a>
### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>1. Télécharger le jeu de données (SEC EDGAR)</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Nous allons télécharger notre jeu de données principal, les derniers dépôts financiers de Nvidia (symbole : <strong>NVDA</strong>). Cette collection de documents formera la base de connaissances non structurées pour notre agent.</p>
    <p style='margin:0 8px 0 0;font-weight:600;color:#000;'>Nous téléchargerons :</p>
    <ul style='margin:8px 0 0 18px;'>
      <li>Le dernier Rapport Annuel (10-K)</li>
      <li>Les quatre derniers Rapports Trimestriels (10-Q)</li>
      <li>Un rapport 8-K récent (pour les événements significatifs)</li>
      <li>La dernière Déclaration de Procuration (DEF 14A)</li>
    </ul>
  </div>
</div>

In [6]:
# Initialize the downloader. Your company name and email are required by the SEC EDGAR API.
dl = Downloader("alpgo", "yann.vernaz@alpgo.com")
COMPANY_TICKER = "NVDA" #"GOOG", "MSFT"

print(f"Starting download of SEC filings for {COMPANY_TICKER}...")

# Get 10-K filings
dl.get("10-K", COMPANY_TICKER, limit=1)
print("Downloaded 10-K filing.")

# Get 10-Q filings
dl.get("10-Q", COMPANY_TICKER, limit=4)
print("Downloaded 10-Q filings.")

# Get 8-K filings
dl.get("8-K", COMPANY_TICKER, limit=1)
print("Downloaded 8-K filing.")

# Get Proxy Statements
dl.get("DEF 14A", COMPANY_TICKER, limit=1)
print("Downloaded DEF 14A filing.")

print("\nAll downloads complete.")

Starting download of SEC filings for NVDA...
Downloaded 10-K filing.
Downloaded 10-Q filings.
Downloaded 8-K filing.
Downloaded DEF 14A filing.

All downloads complete.


<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Résultats</h2>
    <p style='margin:0 0 8px 0;color:#000;'>On a créé un répertoire nommé <code>sec-edgar-filings/</code> avec un dossier pour <strong>NVDA</strong>, qui contient des sous-dossiers pour chaque type de dépôt (10-K, 10-Q, etc.).</p>
    <p style='margin:0 8px 0 0;font-weight:600;color:#000;'>Points clés :</p>
    <ul style='margin:8px 0 0 18px;color:#000;'>
      <li>Les fichiers ont été téléchargés dans le dossier approprié</li>
      <li>Vérifier la présence des fichiers <code>full-submission.txt</code></li>
      <li>Prêt pour l'étape d'ingestion</li>
    </ul>
  </div>
</div>

Répertorions les fichiers que nous avons téléchargés afin de nous assurer que tout est en place avant de passer à la phase d'ingestion.

In [10]:
DATA_PATH = Path(f"sec-edgar-filings/{COMPANY_TICKER}/")
all_files = list(DATA_PATH.rglob("full-submission.txt"))

print(f"Found {len(all_files)} submission files to process:")
for file in all_files:
    print(f"- {file}")

Found 7 submission files to process:
- sec-edgar-filings/NVDA/10-K/0001045810-25-000023/full-submission.txt
- sec-edgar-filings/NVDA/10-Q/0001045810-25-000230/full-submission.txt
- sec-edgar-filings/NVDA/10-Q/0001045810-25-000209/full-submission.txt
- sec-edgar-filings/NVDA/10-Q/0001045810-25-000116/full-submission.txt
- sec-edgar-filings/NVDA/10-Q/0001045810-24-000316/full-submission.txt
- sec-edgar-filings/NVDA/8-K/0001045810-25-000228/full-submission.txt
- sec-edgar-filings/NVDA/DEF 14A/0001045810-25-000095/full-submission.txt


<a id="donnees-relationnelles"></a>
### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>2. Jeu de données structuré (relationnel)</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Pour donner des informations à notre agent <code>Analyst</code>, nous avons besoin d'un jeu de données structuré.</p>
    <p style='margin:0 8px 0 0;font-weight:600;color:#000;'>Objectifs :</p>
    <ul style='margin:8px 0 0 18px;color:#000;'>
      <li>Générer un fichier CSV avec revenus et indicateurs clés</li>
      <li>Vérifier la structure et les types</li>
      <li>Préparer le dataset pour l'agent Analyst</li>
    </ul>
  </div>
</div>

In [14]:
# Load financial data from Excel file
excel_file = "nvidia_financials_2020_2024.xlsx"
df = pd.read_excel(excel_file)

# Save to CSV
CSV_PATH = "revenue_summary.csv"
df.to_csv(CSV_PATH, index=False)

print("Created relational dataset from NVIDIA financial data:")
print(df.head())
print(f"\nDataset shape: {df.shape}")

Created relational dataset from NVIDIA financial data:
   year quarter  revenue_usd_billions  net_income_usd_billions
0  2020      Q1                  3.11                     0.95
1  2020      Q2                  3.08                     0.92
2  2020      Q3                  3.87                     0.62
3  2020      Q4                  4.73                     1.34
4  2021      Q1                  5.00                     1.46

Dataset shape: (20, 4)


<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Résultats</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Nous avons créé un fichier <code>revenue_summary.csv</code>. La sortie affiche les premières lignes de notre table, confirmant la structure de notre simple base de données relationnelle. Ceci complète la phase d'acquisition des données. Les données non structurées (dépôts HTML) et structurées (CSV) sont prêtes pour l'ingestion.</p>
    <p style='margin:0 8px 0 0;font-weight:600;color:#000;'>Points clés :</p>
    <ul style='margin:8px 0 0 18px;color:#000;'>
      <li>Le fichier <code>revenue_summary.csv</code> a été créé avec succès</li>
      <li>Vérifier la présence des fichiers <code>full-submission.txt</code> dans <code>sec-edgar-filings/</code></li>
      <li>Prêt pour l'étape d'ingestion</li>
    </ul>
  </div>
</div>

---

<a id="prochaines-etapes"></a>
### Prochaines étapes

La phase d'acquisition des données est terminée ! Les fichiers/données suivants sont maintenant prêts :
- Dépôts SEC EDGAR stockés dans `sec-edgar-filings/NVDA/`
- Données financières structurées dans `revenue_summary.csv`

Dans un RAG agentique, le noyau de connaissance ou `knowledge core` représente généralement :

- La base de données ou le graphe de connaissances structuré.
- Les informations fiables et validées qui servent de référence.

**[Passez au notebook Phase 1 (Construction du noyau de connaissances)](phase_1_knowledge_core.ipynb)**